In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
%matplotlib inline

In [2]:
# api_key = 'U93z8Z-2erhNJgpJ1mNy'
# api_url = "https://www.quandl.com/api/v3/datasets/WIKI/GOOGL/data.json?api_key={}".format(api_key)
# params = {'limit': 1000}
# response = requests.get(api_url, params=params)
# m_dict = response.json()
df = pd.read_csv('daily_TSLA.csv', parse_dates=['timestamp']).sort_values('timestamp').set_index('timestamp')
df.head()

,open,high,low,close,volume
timestamp,,,,,
2010-06-29,19.00,25.0000,17.54,23.89,18766300
2010-06-30,25.79,30.4192,23.30,23.83,17187100
2010-07-01,25.00,25.9200,20.27,21.96,8218800
2010-07-02,23.00,23.1000,18.71,19.20,5139800
2010-07-06,20.00,20.0000,15.83,16.11,6866900


In [3]:
def moving_average(df, swindow, lwindow):
    sm = df.close.rolling(swindow).mean()
    lo = df.close.rolling(lwindow).mean()
    buy = sm > lo
    sell = sm < lo
    return buy,sell

In [4]:
buy, sell = moving_average(df, 50, 200)

In [5]:
portfolio = 100
def backtest(df, buy, sell, capital, ini_shares, number = 10):
    portfolio = ini_shares
    cash = capital
    df['buy'] = buy
    df['sell'] = sell
    trades = []
    for index, row in df[(df['buy'] == True) | (df['sell'] == True)].iterrows():
        trade = {}
        trade['price'] = row['close']
        trade['timestamp'] = index
        if(row['buy']):
            portfolio += number
            cash = cash - number*(row['close'])
            trade['portfolio'] = portfolio
            trade['cash'] = cash
            trade['type'] = 'buy'
        elif(row['sell']):
            portfolio -= number
            cash = cash + number*(row['close'])
            trade['portfolio'] = portfolio
            trade['cash'] = cash
            trade['type'] = 'sell'
        trades.append(trade)
    trades_df = pd.DataFrame(trades).set_index('timestamp')
    return trades_df

In [6]:
backtest(df, buy, sell, 1000000, portfolio, 10)

,cash,portfolio,price,type
timestamp,,,,
2011-04-12,999753.500,110,24.65,buy
2011-04-13,999504.200,120,24.93,buy
2011-04-14,999252.800,130,25.14,buy
2011-04-15,998997.000,140,25.58,buy
2011-04-18,998746.700,150,25.03,buy
2011-04-19,998495.100,160,25.16,buy
2011-04-20,998237.600,170,25.75,buy
2011-04-21,997970.200,180,26.74,buy
2011-04-25,997706.300,190,26.39,buy
